In [1]:
import pandas as pd, dill as pickle, os
from utils.calculateConstants import *
from utils.util import is_laptop
from datetime import date

In [2]:
folder_path =  'E:/Skóli/HÍ/Vélaverkfræði Master HÍ/Lokaverkefni/Data/' if not is_laptop() else 'D:\Skoli\Mastersverkefni\lokaverkefni_vel'
stationsLonLatXY_path = folder_path + 'Measured/stationsLonLatXY.pkl'
measured_path = folder_path + 'Measured/combined_10min/' + max(os.listdir(folder_path + 'Measured/combined_10min/'), key = lambda f: os.path.getmtime(folder_path + 'Measured/combined_10min/' + f))
reanalysis_path = folder_path + 'Reanalysis/' + max([file for file in os.listdir(folder_path + 'Reanalysis/') if file.endswith('.feather')], key = lambda f: os.path.getmtime(folder_path + 'Reanalysis/' + f))

In [3]:
def addXYtoMeasured(row, stationsLonLatXY):
    station = row.stod
    values = stationsLonLatXY.get(station, [None, None, None, None])
    return pd.Series(values[2:])

In [4]:
with open(stationsLonLatXY_path, 'rb') as f:
    stationsLonLatXY = pickle.load(f)

In [5]:
decimal_places = 4

In [6]:
measured_df = pd.read_feather(measured_path)
reanalysis_df = pd.read_feather(reanalysis_path)

In [8]:
measured_df = measured_df.rename(columns = {'timi':'time'})
measured_df[['X', 'Y']] = measured_df.apply(addXYtoMeasured, args=[stationsLonLatXY], axis = 1)
measured_df = measured_df.round(decimal_places)

In [9]:
reanalysis_df = reanalysis_df.rename(columns = {'lat':'Y', 'lon':'X', 'Wind speed':'ws', 'Wind direction': 'wd', 'Pressure':'p', 'Temperature':'t'})
reanalysis_df = reanalysis_df.drop_duplicates(subset=['X', 'Y', 'time', 'height_level'])
reanalysis_df = reanalysis_df.pivot(index = ['X', 'Y', 'time'], columns = 'height_level')
reanalysis_df = reanalysis_df.drop(columns='yr_month')
reanalysis_df.columns = [f'{col[0]}_{col[1]}' for col in reanalysis_df.columns]
reanalysis_df = reanalysis_df.reset_index()
reanalysis_df.time = pd.to_datetime(reanalysis_df.time)
reanalysis_df = reanalysis_df.round(4)
reanalysis_df[['Ri_01', 'Ri_12', 'Ri_02']] = reanalysis_df.apply(rowRichardson, axis = 1).apply(pd.Series)
reanalysis_df[['N_01', 'N_12', 'N_02']] = reanalysis_df.apply(rowBruntVaisala, axis = 1).apply(pd.Series)

In [10]:
merged_df = pd.merge(measured_df, reanalysis_df, on = ['time', 'X', 'Y'], how = 'inner')
merged_df[['N_01', 'N_12', 'N_02']] = merged_df[['N_01', 'N_12', 'N_02']].map(lambda x: (x.real, x.imag))
merged_df = merged_df.drop(['fsdev', 'dsdev'], axis = 1)
merged_df.to_feather(folder_path + 'Combined/' + f'merged_{date.today()}.feather')

# To go other direction with complex numbers
# df[['column2', 'column3']] = df[['column2', 'column3']].map(lambda x: complex(*x))
